# **Fishing Dataset Folium Map for PS and T**

In this notebook we took the max of each fishing method by month and plotted it on a folium map. We had to experiment with alot of different maps and palettes to extract something useful and visiable from the data. Additionally, we faced some challenges in that our region was not the best for some of the fishing types so often the region had very little data in it. This was also due in part by the fact that most of the ocean is not going to have much fishing in it.

**Note:** In order to see the maps you might have to run the code blocks below as it seems the maps dont save overtime on colab.

**Note:** idential methods to this were followed in another notebook for the other fishing methods



# **Necessary Downloads**

In [ ]:
pip install geemap

     |████████████████████████████████| 450kB 17.6MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 1.6MB 40.4MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 225kB 50.8MB/s 
     |████████████████████████████████| 1.3MB 44.9MB/s 
     |████████████████████████████████| 5.1MB 38.7MB/s 
     |████████████████████████████████| 143kB 48.5MB/s 
     |████████████████████████████████| 1.2MB 37.6MB/s 
     |████████████████████████████████| 102kB 8.0MB/s 
     |████████████████████████████████| 122kB 29.9MB/s 
     |████████████████████████████████| 378kB 41.9MB/s 
     |████████████████████████████████| 552kB 39.7MB/s 
     |████████████████████████████████| 122kB 43.8MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-cp37-none-any.whl size=37264 sha256=4413affe64ac8e43ef4a1f45ff8

In [ ]:
!earthengine authenticate


Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=NlsfNyDftdVwhpRVRtN7WRpdfYUmfCYmqe-CnXU2o9k&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5a8ZdXHZEvipgBvLZDQOgDbuAkP7gLc5OKAKAlQDAr33hGMRMpHyk

Successfully saved authorization token.


In [ ]:
!pip install earthengine-api
!pip install folium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ee
import geemap
ee.Initialize()

import matplotlib.pyplot as plt
import numpy as np

# **Getting the Geometry**

In [ ]:
#from class
import json
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma

# jsonFile = '/content/drive/My Drive/ML/Final_Project/map.geojson' # amys
jsonFile = '/content/drive/My Drive/ECE471/Final_Project/map.geojson' #phil

with open(jsonFile, 'r') as file:
  jsonLoad = json.load(file)

geometry = jsonLoad['features'][0]['geometry']

In [ ]:
geometry

{'coordinates': [[[-75.146484375, 32.32427558887655],
   [-9.31640625, 32.32427558887655],
   [-9.31640625, 46.49839225859763],
   [-75.146484375, 46.49839225859763],
   [-75.146484375, 32.32427558887655]]],
 'type': 'Polygon'}

# **Folium Map**

In [ ]:
from IPython.display import Image
import folium
from folium import plugins

In [ ]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Set visualization parameters.
DLVis = {
  min: 0.0,
  max: 50.0,
}

FGVis = {
  min: 0.0,
  max: 50.0,
}

OFVis = {
  min: 0.0,
  max: 50.0,
}

PSVis = {
  min: 0.0,
  max: 50.0,
}

SJVis = {
  min: 0.0,
  max: 50.0,
}

trawlersVis = {
  min: 0.0,
  max: 50.0,
}

points = [(geometry['coordinates'][0][0][1],geometry['coordinates'][0][0][0]),(geometry['coordinates'][0][2][1],geometry['coordinates'][0][2][0])]

#Map.setCenter(20.3, -20.39, 2)
#Map.addLayer(sstMean, visParams, 'Sea Surface Temperature')

# Create a folium map object.
my_map = folium.Map(location=[20.3, -20.39], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add layers
Map = geemap.Map(center=[20.3, -20.39], zoom=8)

#Map.setCenter(16.201, 36.316, 7);
my_map.add_ee_layer(DL.max(), trawlersVis, 'Drifting Longlines');
my_map.add_ee_layer(FG.max(), trawlersVis, 'Fixed Gear');
my_map.add_ee_layer(OF.max(), trawlersVis, 'Other Fishing');
my_map.add_ee_layer(PS.max(), trawlersVis, 'Purse Seines');
my_map.add_ee_layer(SJ.max(), trawlersVis, 'Squid Jigger');
my_map.add_ee_layer(trawlers.max(), trawlersVis, 'Trawlers');


# my_map.add_ee_layer(ic_DL_Max, visParams1, 'drifting_longlines')

# my_map.add_ee_layer(ic_FG_Max, visParams1, 'fixed_gear')

# my_map.add_ee_layer(ic_OF_Max, visParams1, 'other_fishing')

# my_map.add_ee_layer(ic_PS_Max, visParamsPS, 'purse_seines')

# my_map.add_ee_layer(ic_T_Max, visParams1, 'trawlers')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
#my_map.fit_bounds(geometry['coordinates'][0])
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(my_map)

display(my_map)

NameError: ignored

In [ ]:
datasetPS1 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-01-01', '2016-01-31')).select('purse_seines')
datasetPS2 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-02-01', '2016-02-28')).select('purse_seines')
datasetPS3 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-03-01', '2016-03-31')).select('purse_seines')
datasetPS4 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-04-01', '2016-04-30')).select('purse_seines')
datasetPS5 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-05-01', '2016-05-31')).select('purse_seines')
datasetPS6 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-06-01', '2016-06-30')).select('purse_seines')
datasetPS7 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-07-01', '2016-07-31')).select('purse_seines')
datasetPS8 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-08-01', '2016-08-31')).select('purse_seines')
datasetPS9 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-09-01', '2016-09-30')).select('purse_seines')
datasetPS10 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-10-01', '2016-10-31')).select('purse_seines')
datasetPS11 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-11-01', '2016-11-30')).select('purse_seines')
datasetPS12 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-12-01', '2016-12-31')).select('purse_seines')

In [ ]:
# Set visualization parameters.
PSVis = {
  min: 0.0,
  max: 50.0,
  "palette": ['8ab4f8', 'FF0000'],
  "opacity": 0.7
}


points = [(geometry['coordinates'][0][0][1],geometry['coordinates'][0][0][0]),(geometry['coordinates'][0][2][1],geometry['coordinates'][0][2][0])]

#Map.setCenter(20.3, -20.39, 2)
#Map.addLayer(sstMean, visParams, 'Sea Surface Temperature')

# Create a folium map object.
my_map = folium.Map(location=[20.3, -20.39], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
#basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add layers
Map = geemap.Map(center=[20.3, -20.39], zoom=8)

#Map.setCenter(16.201, 36.316, 7);
my_map.add_ee_layer(datasetPS1.max(), PSVis, 'Purse Seines Jan');
my_map.add_ee_layer(datasetPS2.max(), PSVis, 'Purse Seines Feb');
my_map.add_ee_layer(datasetPS3.max(), PSVis, 'Purse Seines Mar');
my_map.add_ee_layer(datasetPS4.max(), PSVis, 'Purse Seines Apr');
my_map.add_ee_layer(datasetPS5.max(), PSVis, 'Purse Seines May');
my_map.add_ee_layer(datasetPS6.max(), PSVis, 'Purse Seines Jun');
my_map.add_ee_layer(datasetPS7.max(), PSVis, 'Purse Seines Jul');
my_map.add_ee_layer(datasetPS8.max(), PSVis, 'Purse Seines Aug');
my_map.add_ee_layer(datasetPS9.max(), PSVis, 'Purse Seines Sep');
my_map.add_ee_layer(datasetPS10.max(), PSVis, 'Purse Seines Oct');
my_map.add_ee_layer(datasetPS11.max(), PSVis, 'Purse Seines Nov');
my_map.add_ee_layer(datasetPS12.max(), PSVis, 'Purse Seines Dec');

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
#my_map.fit_bounds(geometry['coordinates'][0])
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.01).add_to(my_map)
#folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(my_map)

display(my_map)

In [ ]:
datasetT1 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-01-01', '2016-01-31')).select('trawlers')
datasetT2 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-02-01', '2016-02-28')).select('trawlers')
datasetT3 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-03-01', '2016-03-31')).select('trawlers')
datasetT4 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-04-01', '2016-04-30')).select('trawlers')
datasetT5 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-05-01', '2016-05-31')).select('trawlers')
datasetT6 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-06-01', '2016-06-30')).select('trawlers')
datasetT7 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-07-01', '2016-07-31')).select('trawlers')
datasetT8 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-08-01', '2016-08-31')).select('trawlers')
datasetT9 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-09-01', '2016-09-30')).select('trawlers')
datasetT10 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-10-01', '2016-10-31')).select('trawlers')
datasetT11 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-11-01', '2016-11-30')).select('trawlers')
datasetT12 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-12-01', '2016-12-31')).select('trawlers')

In [ ]:
# Set visualization parameters.
TVis = {
  min: 0.0,
  max: 50.0,
  "palette": ['8ab4f8', 'FF0000'],
  "opacity": 0.7
}


points = [(geometry['coordinates'][0][0][1],geometry['coordinates'][0][0][0]),(geometry['coordinates'][0][2][1],geometry['coordinates'][0][2][0])]

#Map.setCenter(20.3, -20.39, 2)
#Map.addLayer(sstMean, visParams, 'Sea Surface Temperature')

# Create a folium map object.
my_map = folium.Map(location=[20.3, -20.39], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
#basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add layers
Map = geemap.Map(center=[20.3, -20.39], zoom=8)

#Map.setCenter(16.201, 36.316, 7);
my_map.add_ee_layer(datasetT1.max(), TVis, 'Trawlers Jan');
my_map.add_ee_layer(datasetT2.max(), TVis, 'Trawlers Feb');
my_map.add_ee_layer(datasetT3.max(), TVis, 'Trawlers Mar');
my_map.add_ee_layer(datasetT4.max(), TVis, 'Trawlers Apr');
my_map.add_ee_layer(datasetT5.max(), TVis, 'Trawlers May');
my_map.add_ee_layer(datasetT6.max(), TVis, 'Trawlers Jun');
my_map.add_ee_layer(datasetT7.max(), TVis, 'Trawlers Jul');
my_map.add_ee_layer(datasetT8.max(), TVis, 'Trawlers Aug');
my_map.add_ee_layer(datasetT9.max(), TVis, 'Trawlers Sep');
my_map.add_ee_layer(datasetT10.max(), TVis, 'Trawlers Oct');
my_map.add_ee_layer(datasetT11.max(), TVis, 'Trawlers Nov');
my_map.add_ee_layer(datasetT12.max(), TVis, 'Trawlers Dec');

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
#my_map.fit_bounds(geometry['coordinates'][0])
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.01).add_to(my_map)
#folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(my_map)

display(my_map)